In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# set a style 
sns.set(style="whitegrid")


# to get the session 
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# to load the data
customers     = session.table("CUSTOMERS").to_pandas()
subscriptions = session.table("SUBSCRIPTIONS").to_pandas()
invoices      = session.table("INVOICES").to_pandas()
charges       = session.table("CHARGES").to_pandas()
plan_prices   = session.table("PLAN_PRICES").to_pandas()
utm_sources   = session.table("UTM_SOURCES").to_pandas()


In [ ]:
# to check null values
def check_nulls(df, name):
    print(f"\n=== {name} ===")
    print(df.isna().sum().sort_values(ascending=False))

check_nulls(customers, "CUSTOMERS")
check_nulls(subscriptions, "SUBSCRIPTIONS")
check_nulls(invoices, "INVOICES")
check_nulls(charges, "CHARGES")
check_nulls(plan_prices, "PLAN_PRICES")
check_nulls(utm_sources, "UTM_SOURCES")

In [ ]:
# to check the primary keys
def check_pk(df, cols, name):
    dups = df.duplicated(subset=cols).sum()
    print(f"[{name}] duplicates by {cols}: {dups}")

check_pk(customers, ["CUSTOMER_ID"], "CUSTOMERS")
check_pk(plan_prices, ["PLAN_ID"], "PLAN_PRICES")
check_pk(subscriptions, ["SUBSCRIPTION_ID"], "SUBSCRIPTIONS")
check_pk(invoices, ["INVOICE_ID"], "INVOICES")
check_pk(charges, ["CHARGE_ID"], "CHARGES")
check_pk(utm_sources, ["UTM_ID"], "UTM_SOURCES")


In [ ]:
# Distribution of Invoice Amount Paid
plt.figure(figsize=(8,4))
sns.histplot(invoices["AMOUNT_PAID"].dropna()/100.0, bins=30)
plt.title("Distribution of Invoice Amount Paid (USD)")
plt.xlabel("Invoice Amount Paid (USD)")
plt.show()

#  Distribution of Charge Amounts
plt.figure(figsize=(8,4))
sns.histplot(charges["AMOUNT"].dropna()/100.0, bins=30)
plt.title("Distribution of Charge Amounts (USD)")
plt.xlabel("Charge Amount (USD)")
plt.show()

In [ ]:
print("\nInvoice amounts summary (USD):")
print((invoices["AMOUNT_PAID"]/100.0).describe())

print("\nCharges amounts summary (USD):")
print((charges["AMOUNT"]/100.0).describe())

print("\nDays past due summary:")
print(invoices["DAYS_PAST_DUE"].describe())

In [ ]:
# Invoices without Customer
orphan_invoices = invoices[~invoices["CUSTOMER_ID"].isin(customers["CUSTOMER_ID"])]
print(f"Invoices without valid customer: {len(orphan_invoices)}")

# Invoices without Subscription
orphan_invoices_subs = invoices[~invoices["SUBSCRIPTION_ID"].isin(subscriptions["SUBSCRIPTION_ID"])]
print(f"Invoices without valid subscription: {len(orphan_invoices_subs)}")

# Charges without Invoice
orphan_charges = charges[~charges["INVOICE_ID"].isin(invoices["INVOICE_ID"])]
print(f"Charges without valid invoice: {len(orphan_charges)}")


In [ ]:
# check for negative values
print("Negative invoice amounts:", (invoices["AMOUNT_PAID"] < 0).sum())
print("Negative charge amounts:", (charges["AMOUNT"] < 0).sum())

# check currencys
print("Unexpected currencies (invoices):", invoices[~invoices["CURRENCY"].isin(["usd","USD"])]["CURRENCY"].unique())

# check status
print("Invoice statuses:", invoices["STATUS"].unique())
print("Subscription statuses:", subscriptions["STATUS"].unique())
print("Charge statuses:", charges["STATUS"].unique())

In [ ]:
# Invoice period_end >= period_start
invalid_periods = (invoices["PERIOD_END"] < invoices["PERIOD_START"]).sum()
print("Invoices with period_end < period_start:", invalid_periods)

# Paid_date >= due_date
mask = invoices["PAID_DATE"].notna() & invoices["DUE_DATE"].notna()
invalid_paid = (invoices.loc[mask, "PAID_DATE"] < invoices.loc[mask, "DUE_DATE"]).sum()
print("Invoices with paid_date < due_date:", invalid_paid)

# Trial_end >= trial_start
mask = subscriptions["TRIAL_START"].notna() & subscriptions["TRIAL_END"].notna()
invalid_trials = (subscriptions.loc[mask, "TRIAL_END"] < subscriptions.loc[mask, "TRIAL_START"]).sum()
print("Subscriptions with trial_end < trial_start:", invalid_trials)

In [ ]:
select * from invoices where paid_date < due_date limit 10;

In [ ]:
print("Invoices date range:", invoices["PERIOD_START"].min(), "to", invoices["PERIOD_END"].max())
print("Subscriptions date range:", subscriptions["START_DATE"].min(), "to", subscriptions["CANCEL_DATE"].max())

# timeline of the number of invoices
invoices["MONTH"] = pd.to_datetime(invoices["PERIOD_END"]).dt.to_period("M")
inv_by_month = invoices.groupby("MONTH")["INVOICE_ID"].count()

inv_by_month.plot(kind="bar", figsize=(10,4), title="Invoices per Month")
plt.show()


In [ ]:
# Boxplot paid values
plt.figure(figsize=(6,4))
sns.boxplot(x=invoices["AMOUNT_PAID"]/100.0)
plt.title("Invoice Amount Paid (USD)")
plt.show()

# top 5 overdue invoices
print("Top 5 invoices by days_past_due:")
print(invoices[["CUSTOMER_ID","INVOICE_ID","DAYS_PAST_DUE"]].sort_values("DAYS_PAST_DUE", ascending=False).head())



In [ ]:
SELECT * from invoices where invoice_id = 'in_2bwz0lgfdw6gud'

In [ ]:
# Distribution of Days Past Due
plt.figure(figsize=(6,4))
sns.boxplot(x=invoices["DAYS_PAST_DUE"].dropna())
plt.title("Distribution of Days Past Due")
plt.show()